# Dinosaur Names Generator with RNNs 
---

First let's import the libraries we'll use:

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [4]:
dino_file = open('datasets/dino.txt', 'r')
dinos = dino_file.read()
dino_file.close()
print(dinos[:99])

aachenosaurus
aardonyx
abelisaurus
abrictosaurus
abrosaurus
abydosaurus
acantholipan
acanthopholis



## Data Preprocessing:

In [5]:
dino_list = dinos.split('\n')
print(dino_list[3:6])
print(f'The dataset contains {len(dino_list)} exemples')

['abrictosaurus', 'abrosaurus', 'abydosaurus']
The dataset contains 1533 exemples


First let's see all the chars that figure in the names:

In [6]:
all_chars = []
for dino in dino_list:
    splitted = list(dino)
    for c in splitted:
        if c not in all_chars:
            all_chars.append(c)
print(all_chars)

['a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'k', 'g', 'f', 'j', 'w', 'z', 'q', '_']


So in addition to alphabet chars, the underscore '_' does also figure in and should be counted later, next we'll append each training sample with a start and end character padding to a maximum length:
But before let's see the longest dinosaur name in our dataset:

In [7]:
max_len = 0
for dino in dino_list:
    lng = len(dino)
    if lng > max_len:
        max_len = lng
print(f'longest name contains {max_len} characters')

longest name contains 23 characters


Now let's pad the names and add the start/end chars:
But first let's create a simple padding function:

In [8]:
def padname(name, pad_char, maxl):
    to_add = maxl - len(name)
    if to_add <= 0:
        return name
    to_append = ''.join([pad_char] * to_add)
    return name + to_append

padname('Dino', '$', 7) # exemple

'Dino$$$'

In [9]:
start_char = '~'
end_char = '$'
dino_list = [start_char + padname(dino_name.lower(), end_char, max_len) + end_char for dino_name in dino_list]
print(dino_list[0:5])

['~aachenosaurus$$$$$$$$$$$', '~aardonyx$$$$$$$$$$$$$$$$', '~abelisaurus$$$$$$$$$$$$$', '~abrictosaurus$$$$$$$$$$$', '~abrosaurus$$$$$$$$$$$$$$']


Next, we'll split dino names into charcaters to form our dataset: 

In [10]:
dataset_X = [list(dino) for dino in dino_list]
dataset_y = []
for l in dataset_X:
    y_label = l[1:] + [end_char]
    dataset_y.append(y_label)
    
print(dataset_X[0]) # first sample
print(dataset_y[0]) # first label

['~', 'a', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$']
['a', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$', '$']


We'll feed the RNN with one-hot representation of characters and to do so, we'll need to create a dictionnary mapping each char in the dataset to an integer which will be converted to a one hot later:

In [11]:
char_to_int = {chr(x): x - ord('a') for x in range(ord('a'), ord('z') + 1)}
char_to_int[start_char] = 26
char_to_int[end_char] = 27
char_to_int['_'] = 28
print(char_to_int)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '~': 26, '$': 27, '_': 28}


Now let's get the inverted dictionnay:

In [12]:
int_to_char = {v: k for k, v in char_to_int.items()}
print(int_to_char)

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: '~', 27: '$', 28: '_'}


Using the dictionary above, we can now implement a fucntion that returns the one-hot vector of a character:

In [13]:
def one_hot(c):
    order = char_to_int[c]
    return keras.utils.to_categorical(order, num_classes = len(char_to_int))

def one_hot_list(l): # ret: numpy array
    ret = []
    for c in l:
        ret.append(one_hot(c))
    return np.array(ret)

print(one_hot('f'))
print('-------------------------------')
print(one_hot_list(['a', 'b', 'c']))

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
-------------------------------
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]


In [14]:
Tx = len(dino_list[0])
n_samples = len(dino_list)
input_vec_shape = (len(one_hot('a')), )

print(f'Sequence length = {Tx}')
print(f'Number of samples = {n_samples}')
print(f'Input vector shape = {input_vec_shape}')


X, y = np.zeros((n_samples, Tx, input_vec_shape[0])), np.zeros((n_samples, Tx, input_vec_shape[0]))

for i in range(n_samples):
    X_hot = one_hot_list(dataset_X[i])
    y_hot = one_hot_list(dataset_y[i])
    X[i, :, :] = X_hot
    y[i, :, :] = y_hot
    
print('--')
print(f'X shape is {X.shape}')
print(f'y shape is {y.shape}')

Sequence length = 25
Number of samples = 1533
Input vector shape = (29,)
--
X shape is (1533, 25, 29)
y shape is (1533, 25, 29)


Now that our dataset is preprocessed, let's create our model and compile it:
### Model and Training
---

In [16]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[None, 29]),
    keras.layers.LSTM(120, return_sequences = True),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(29, activation = 'softmax'),
])

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 120)         72000     
_________________________________________________________________
dense_2 (Dense)              (None, None, 64)          7744      
_________________________________________________________________
dense_3 (Dense)              (None, None, 29)          1885      
Total params: 81,629
Trainable params: 81,629
Non-trainable params: 0
_________________________________________________________________


### Training

In [19]:
model.fit(X, y, batch_size = n_samples, epochs=10, verbose = 1, shuffle=False)

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 1.8669 - acc: 0.5271
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 1.8884 - acc: 0.5271
Epoch 3/10
1/1 [==============================] - 0s 1ms/step - loss: 1.8790 - acc: 0.5271
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 1.8451 - acc: 0.5271
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 1.7957 - acc: 0.5271
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 1.7410 - acc: 0.5271
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 1.6907 - acc: 0.5271
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 1.6519 - acc: 0.5274
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 1.6284 - acc: 0.5274
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 1.6213 - acc: 0.5280


For more conveniance, let's save the model so we can load it (with the weights) whenever we want: 

In [170]:
model.save('./models/RNN.h5')

# Model Loading & Prediction:

In [15]:
model = keras.models.load_model('./models/RNN.h5')
ex_loss, ex_acc = model.evaluate(X, y)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(X, y, batch_size = n_samples, epochs=5, verbose = 1, shuffle=False)

now_loss, now_acc = model.evaluate(X, y)
if now_loss < ex_loss:
    model.save('RNN.h5')
    print('model saved !')

48/48 [==============================] - 0s 9ms/step - loss: 0.3204 - acc: 0.8916
Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 0.3204 - acc: 0.8916
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 0.3761 - acc: 0.8765
Epoch 3/5
1/1 [==============================] - 0s 2ms/step - loss: 0.3297 - acc: 0.8899
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 0.3485 - acc: 0.8837
Epoch 5/5
48/48 [==============================] - 0s 9ms/step - loss: 0.3362 - acc: 0.8884


Next, we'll define a function to retrieve the string form of the predicted sequence from its one-hot representation:

In [16]:
def get_str_pred(pred): # pred shape sould be (1x..x1) x 29
    pred = pred.reshape(-1, 29)
    str_pred = []
    for c in pred:
#         int_c = np.random.choice(29, p = c)
        int_c = np.argmax(c)
        
        real_c = int_to_char[int_c]
        str_pred.append(real_c)
    
    return ''.join(str_pred)

x = np.array([X[0, :, :]])
str_pred = get_str_pred(model.predict(x))
print(f'string prediction = {str_pred}')

string prediction = anraenosaurus$$$$$$$$$$$$


Finally, we can implement a generator function that takes a sequence beginning as argument and complete it by recursively predicting the next character and let's try it over some funny names:

In [30]:
def generate_after(seq_beg):
    seq_hot = np.array(one_hot_list(list(seq_beg))).reshape(1, -1, 29)
    pred = model.predict(seq_hot)[0, -1, :].argmax()
    pred = int_to_char[pred]
    if pred != '$':
        return generate_after(seq_beg + pred)
    return seq_beg[1:] # remove the start token

# Let's try some fun names

print(generate_after('~macron').upper()) # Emanuel Macron 
print('--')
print(generate_after('~trudau').upper()) # Justin Trudeau
print('--')
print(generate_after('~trevor').upper()) # Trevor (Noah)
print('--')
print(generate_after('~gru').upper()) # GRU (Gated Recurrent Unit xD)

MACRONOSAURUS
--
TRUDAUDIN
--
TREVORHON
--
GRUSJITASAUR
